<a href="https://colab.research.google.com/github/Harshakiran1244/Movie-Recommendation-Algorithm/blob/main/Movie_Recommendation_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
import pandas as pd
import numpy as np
movies_df = pd.read_csv('/content/tmdb_5000_movies.csv')
credits_df = pd.read_csv('/content/tmdb_5000_credits.csv')

In [83]:
movies_df = movies_df.merge(credits_df,on = 'title')

In [84]:
movies_df_1 = movies_df.copy()

In [85]:
movies_df = movies_df[['cast','title','crew','genres','movie_id','keywords','overview']]

In [86]:
movies_df_2 = movies_df.copy()

In [89]:
movies_df.isnull().sum()

cast        0
title       0
crew        0
genres      0
movie_id    0
keywords    0
overview    0
dtype: int64

In [88]:
movies_df.dropna(inplace = True)

In [90]:
import ast
def convert_(obj):
  a = []
  count = 0
  for i in ast.literal_eval(obj):
    if count != 3:
      count = count + 1
      a.append(i['name'])
    else:
      break
  return a



In [91]:
movies_df['cast'] = movies_df['cast'].apply(convert_)

In [92]:
movies_df['genres'] = movies_df['genres'].apply(convert_)

In [93]:
movies_df['genres']

0               [Action, Adventure, Fantasy]
1               [Adventure, Fantasy, Action]
2                 [Action, Adventure, Crime]
3                     [Action, Crime, Drama]
4       [Action, Adventure, Science Fiction]
                        ...                 
4804               [Action, Crime, Thriller]
4805                       [Comedy, Romance]
4806                [Comedy, Drama, Romance]
4807                                      []
4808                           [Documentary]
Name: genres, Length: 4806, dtype: object

In [94]:
def fetch_director(obj):
  a = []
  for i in ast.literal_eval(obj):
    if i['job'] == 'Director':
      a.append(i['name'])
  return a

In [96]:
movies_df['crew'] = movies_df['crew'].apply(fetch_director)

In [97]:
movies_df['overview'] = movies_df['overview'].apply(lambda x:x.split())

In [98]:
movies_df['overview']

0       [In, the, 22nd, century,, a, paraplegic, Marin...
1       [Captain, Barbossa,, long, believed, to, be, d...
2       [A, cryptic, message, from, Bond’s, past, send...
3       [Following, the, death, of, District, Attorney...
4       [John, Carter, is, a, war-weary,, former, mili...
                              ...                        
4804    [El, Mariachi, just, wants, to, play, his, gui...
4805    [A, newlywed, couple's, honeymoon, is, upended...
4806    ["Signed,, Sealed,, Delivered", introduces, a,...
4807    [When, ambitious, New, York, attorney, Sam, is...
4808    [Ever, since, the, second, grade, when, he, fi...
Name: overview, Length: 4806, dtype: object

In [100]:
movies_df['genres'] = movies_df['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies_df['cast'] = movies_df['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies_df['crew'] = movies_df['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

In [101]:
movies_df['tags'] = movies_df['overview'] + movies_df['genres'] + movies_df['keywords'] + movies_df['cast'] + movies_df['crew']

In [104]:
new_movies_df = movies_df[['movie_id','title','tags']]

In [106]:
new_movies_df_2 = new_movies_df.copy()

In [109]:
new_movies_df['tags'] = new_movies_df['tags'].apply(lambda x:" ".join(x))

<ipython-input-109-6cafdaff6fe3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movies_df['tags'] = new_movies_df['tags'].apply(lambda x:" ".join(x))


In [112]:
new_movies_df = new_movies_df['tags'].apply(lambda x:x.lower())

In [115]:
new_movies_df_2['tags'] = new_movies_df_2['tags'].apply(lambda x:" ".join(x))

In [118]:
new_movies_df_2['tags'] = new_movies_df_2['tags'].apply(lambda x:x.lower())

In [119]:
new_movies_df_2

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."
...,...,...,...
4804,9367,El Mariachi,el mariachi just wants to play his guitar and ...
4805,72766,Newlyweds,a newlywed couple's honeymoon is upended by th...
4806,231617,"Signed, Sealed, Delivered","""signed, sealed, delivered"" introduces a dedic..."
4807,126186,Shanghai Calling,when ambitious new york attorney sam is sent t...


In [120]:
from sklearn.feature_extraction.text import CountVectorizer

In [121]:
cv = CountVectorizer(max_features = 5000, stop_words = 'english')

In [123]:
vectors = cv.fit_transform(new_movies_df_2['tags']).toarray()

In [125]:
len(vectors)

4806

In [126]:
!pip install nltk

In [127]:
import nltk


In [128]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [129]:
from nltk.stem.porter import PorterStemmer

In [130]:
ps = PorterStemmer()

In [131]:
def stem(text):
  y = []
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)


In [132]:
new_movies_df_2['tags'].apply(stem)

0       in the 22nd century, a parapleg marin is dispa...
1       captain barbossa, long believ to be dead, ha c...
2       a cryptic messag from bond’ past send him on a...
3       follow the death of district attorney harvey d...
4       john carter is a war-weary, former militari ca...
                              ...                        
4804    el mariachi just want to play hi guitar and ca...
4805    a newlyw couple' honeymoon is upend by the arr...
4806    "signed, sealed, delivered" introduc a dedic q...
4807    when ambiti new york attorney sam is sent to s...
4808    ever sinc the second grade when he first saw h...
Name: tags, Length: 4806, dtype: object

In [133]:
from sklearn.metrics.pairwise import cosine_similarity

In [162]:
similarity = cosine_similarity(vectors)

In [140]:
new_movies_df_2[new_movies_df_2['title'] == 'Batman Begins'].index[0]

119

In [178]:
def recommend_(movie):
  movie_index = new_movies_df_2[new_movies_df_2['title'] == movie].index[0]
  distances = similarity[movie_index]
  movies_list = sorted(list(enumerate(distances)),reverse = True,key = lambda x:x[1])[1:6]
  for i in movies_list:
    print(new_movies_df_2.iloc[i[0]].title)

In [150]:
sorted(list(enumerate(similarity[0])),reverse = True,key = lambda x:x[1])[1:6]

[(151, 0.26967994498529685),
 (1444, 0.2480694691784169),
 (4010, 0.23094010767585027),
 (942, 0.22821773229381923),
 (3730, 0.21213203435596426)]

In [180]:
recommend_('The Avengers')

Avengers: Age of Ultron
Allegiant
Iron Man 3
The Blood of Heroes
Captain America: Civil War


In [185]:
import pickle
pickle.dump(new_movies_df_2,open('movies.pkl','wb'))

In [182]:
new_movies_df_2

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."
...,...,...,...
4804,9367,El Mariachi,el mariachi just wants to play his guitar and ...
4805,72766,Newlyweds,a newlywed couple's honeymoon is upended by th...
4806,231617,"Signed, Sealed, Delivered","""signed, sealed, delivered"" introduces a dedic..."
4807,126186,Shanghai Calling,when ambitious new york attorney sam is sent t...


In [189]:
pickle.dump(new_movies_df_2.to_dict(),open('movie_dict.pkl','wb'))

In [193]:
new_movies_df_2['title'].values

array(['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre',
       ..., 'Signed, Sealed, Delivered', 'Shanghai Calling',
       'My Date with Drew'], dtype=object)

In [195]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [194]:
similarity

array([[1.        , 0.15389675, 0.0860663 , ..., 0.        , 0.        ,
        0.        ],
       [0.15389675, 1.        , 0.08830216, ..., 0.03721615, 0.        ,
        0.        ],
       [0.0860663 , 0.08830216, 1.        , ..., 0.03121953, 0.        ,
        0.        ],
       ...,
       [0.        , 0.03721615, 0.03121953, ..., 1.        , 0.08000711,
        0.02782074],
       [0.        , 0.        , 0.        , ..., 0.08000711, 1.        ,
        0.05638839],
       [0.        , 0.        , 0.        , ..., 0.02782074, 0.05638839,
        1.        ]])